In [1]:
# =============================================================================
# BLOCK 0: Pre-Code Blocks
# =============================================================================
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
# =============================================================================
# BLOCK 1: Install required libraries
# =============================================================================
!pip install pandas geopy vaderSentiment faiss-cpu sentence-transformers
# !pip install transformers einops accelerate langchain
# !pip install llama_index
# !pip install llama-index-embeddings-langchain
# !pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
# !pip install -U bitsandbytes


!pip install transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers #Embedding
!pip install llama_index
!pip install llama-index-embeddings-langchain
!pip install llama-index-llms-huggingface

!pip install llama-index-embeddings-huggingface

In [3]:
# =============================================================================
# BLOCK 2: Import libraries
# =============================================================================
import os, re, json, tarfile, pickle
import pandas as pd
import numpy as np
from transformers import pipeline, AutoTokenizer
from google.colab import drive
import faiss
from sentence_transformers import SentenceTransformer

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine

import csv

In [4]:
# =============================================================================
# BLOCK 3: Mount Google Drive
# =============================================================================
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# =============================================================================
# BLOCK 4: Load the sentiment analysis model & tokenizer (for existing processing)
# =============================================================================
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
classifier = pipeline("sentiment-analysis", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Device set to use cuda:0


In [6]:
# =============================================================================
# BLOCK 5: Define path to tweets folder in Drive (for your tweets dataset)
# =============================================================================
tweets_csv_folder = "/content/drive/MyDrive/alaska_vaccine_project/tweets/csv_tweets"

In [7]:
# =============================================================================
# BLOCK 6: Utility Functions for CSV Loading and Tweet Preprocessing
# =============================================================================
def load_csv_safely(file_path):
    encodings = ["latin-1", "ISO-8859-1", "utf-8"]  # Common encodings to try

    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding, engine="python", on_bad_lines="skip")
            return df  # Return dataframe if successful
        except Exception as e:
            print(f"⚠️ Failed to read {file_path} with {encoding}: {e}")

    print(f"🚨 Skipping {file_path} - Unable to read with known encodings.")
    return None  # Return None if all attempts fail

# Function to load tweets from multiple CSV files in a folder
def load_tweets_from_csv_folder(folder_path):
    all_tweets = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = load_csv_safely(file_path)  # Try to read with multiple encodings

            if df is None:
                continue  # Skip if file couldn't be read

            # Ensure 'description' column exists
            if "description" not in df.columns:
                print(f"Skipping {filename} - No 'description' column found. Available columns: {df.columns.tolist()}")
                continue  # Skip file

            # Use 'user_location' if available, otherwise create an empty column
            location_column = "user_location" if "user_location" in df.columns else None
            if location_column is None:
                df["user_location"] = ""

            # Keep only necessary columns and rename for consistency
            tweets = df[["description", "user_location"]].rename(columns={"description": "text"}).to_dict(orient="records")
            all_tweets.extend(tweets)

    return all_tweets

# Function to check if a location is in Alaska (Handles multiple variations)
def is_in_alaska(location):
    if not isinstance(location, str):
        return False

    alaska_patterns = [
        r"\bAlaska\b",  # Matches "Alaska"
        r"\bAK\b",  # Matches "AK"
        r"Anchorage", r"Sitka", r"Juneau", r"Fairbanks",  # Major cities
        r"\bAK, USA\b", r"\bAlaska, USA\b",  # Common variations
        r"\bAK,\s*\w{2,}\b"  # Matches things like "AK, US" or "AK, Canada"
    ]

    return any(re.search(pattern, location, re.IGNORECASE) for pattern in alaska_patterns)

# Function to classify as rural or urban based on location keywords
def classify_rural_urban(location):
    if not isinstance(location, str):
        return "Unknown"

    rural_areas = {"Bethel", "Nome", "Barrow", "Kotzebue", "Wrangell"}  # Example rural areas
    urban_areas = {"Anchorage", "Fairbanks", "Juneau", "Sitka"}  # Example urban areas

    for city in rural_areas:
        if city.lower() in location.lower():
            return "Rural"
    for city in urban_areas:
        if city.lower() in location.lower():
            return "Urban"
    return "Unknown"


# Function to truncate long tweets to 512 tokens
def truncate_text(text):
    tokens = tokenizer.encode(text, truncation=True, max_length=512)
    return tokenizer.decode(tokens, skip_special_tokens=True)

In [8]:
# =============================================================================
# BLOCK 7: Existing Sentiment Analysis Process (for reference)
# =============================================================================
def process_tweets(file_path, data_source="csv"):
    if data_source == "csv":
        tweets = load_tweets_from_csv_folder(file_path)
    else:
        raise ValueError("Invalid data source. Use 'tar' or 'csv'.")
    if not tweets:
        print("No valid tweets found. Exiting...")
        return pd.DataFrame()
    alaska_tweets = []
    for tweet in tweets:
        text = tweet.get("text", "")
        location = tweet.get("user_location", "")
        if is_in_alaska(location):
            rural_urban = classify_rural_urban(location)
            truncated_text = truncate_text(text)
            sentiment = classifier(truncated_text)[0]["label"]
            alaska_tweets.append({
                "text": truncated_text,
                "location": location,
                "rural_urban": rural_urban,
                "sentiment": sentiment
            })
    return pd.DataFrame(alaska_tweets)

In [9]:
# =============================================================================
# BLOCK 8: Data Preparation & Indexing for Retrieval-Augmented Generation (RAG)
# =============================================================================
def process_tweets_for_indexing(file_path, data_source="csv"):
    if data_source == "csv":
        tweets = load_tweets_from_csv_folder(file_path)
    else:
        raise ValueError("Invalid data source. Use 'tar' or 'csv'.")
    if not tweets:
        print("No valid tweets found for indexing. Exiting...")
        return [], []
    alaska_texts = []
    metadata = []
    for tweet in tweets:
        text = tweet.get("text", "")
        location = tweet.get("user_location", "")
        if is_in_alaska(location):
            truncated_text = truncate_text(text)
            alaska_texts.append(truncated_text)
            metadata.append(tweet)
    return alaska_texts, metadata

tweet_texts, tweet_metadata = process_tweets_for_indexing(tweets_csv_folder, "csv")
print(f"Number of Alaska tweets for indexing: {len(tweet_texts)}")

# Build embeddings using SentenceTransformer and create a FAISS index.
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(tweet_texts, show_progress_bar=True)
embeddings = np.array(embeddings).astype('float32')
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)
print(f"FAISS index built with {faiss_index.ntotal} tweet vectors.")

# Save the index and metadata.
faiss.write_index(faiss_index, "tweets_index.faiss")
with open("tweet_metadata.pkl", "wb") as f:
    pickle.dump({"metadata": tweet_metadata, "texts": tweet_texts}, f)

Number of Alaska tweets for indexing: 1342


Batches:   0%|          | 0/42 [00:00<?, ?it/s]

FAISS index built with 1342 tweet vectors.


In [10]:
# =============================================================================
# BLOCK 9: Craft Queries Aligned with Study Objectives
# =============================================================================
queries = [
    # Vaccine Hesitancy Reasons
    "What are the most common reasons for COVID-19 vaccine hesitancy among Alaskans?",
    "What are the key concerns about COVID-19 vaccine safety and effectiveness in Alaska?",
    "How do past negative experiences with vaccines shape vaccine hesitancy?",
    "What role do family and peer networks play in influencing vaccine hesitancy?",
    "How do rural and urban Alaskans differ in their reasons for vaccine hesitancy?",
    "What are the most common misinformation themes about COVID-19 vaccines in Alaska?",
    "How do Alaskans respond to fact-checking efforts on vaccine misinformation?",
    "How do religious beliefs shape vaccine hesitancy in Alaska?",
    "What cultural factors influence vaccine confidence among Native Alaskan communities?",
    "What are the biggest logistical challenges Alaskans face in getting vaccinated?",
    "What strategies have been most successful in engaging vaccine-hesitant individuals in Alaska?",
    "What do healthcare professionals in Alaska believe are the biggest drivers of vaccine hesitancy?",

    # Misinformation & Conspiracy Theories
    "What are the most common misinformation themes about COVID-19 vaccines in Alaska?",
    "What conspiracy theories about COVID-19 vaccines are being discussed in Alaska?",
    "How do Alaskans respond to fact-checking efforts on vaccine misinformation?",
    "How has vaccine misinformation changed over time in Alaska?",
    "How do social media and traditional news sources differ in spreading vaccine misinformation?",
    "What role do influencers and community leaders play in spreading or countering vaccine misinformation?",
    "What misinformation about vaccine side effects is most commonly discussed in Alaska?",

    # Social & Cultural Factors
    "How do religious beliefs shape vaccine hesitancy in Alaska?",
    "What cultural factors influence vaccine confidence among Native Alaskan communities?",
    "How do vaccine hesitancy attitudes vary between different socio-economic groups in Alaska?",
    "How do Alaskan parents view childhood vaccinations compared to COVID-19 vaccines?",
    "How do rural Alaskans perceive vaccines differently than urban residents?",

    # Barriers to Vaccination (Beyond Social Media)
    "What are the biggest logistical challenges Alaskans face in getting vaccinated?",
    "How do vaccine access issues in rural Alaska compare to urban centers like Anchorage?",
    "What concerns do Alaskans express about the speed of vaccine development and approval?",
    "How do people in Alaska perceive government vaccine mandates or incentives?",
    "What role does transportation, cost, and availability play in vaccine hesitancy?",
    "How has the availability of COVID-19 vaccines changed vaccination behavior in Alaska?",

    # Actionable Insights: Social Media Solutions & Policy Recommendations
    "What strategies have been most successful in engaging vaccine-hesitant individuals in Alaska?",
    "What role should social media platforms play in countering vaccine misinformation in Alaska?",
    "What social media campaigns have been most effective in increasing vaccine confidence in Alaska?",
    "What policies or interventions have been proposed to improve vaccine uptake in Alaska?",
    "What lessons can be learned from successful vaccine campaigns in Alaska?",
    "What incentives or outreach methods have been effective in overcoming vaccine hesitancy in Alaska?",

    # Healthcare Professionals & Stakeholder Insights
    "What do healthcare professionals in Alaska believe are the biggest drivers of vaccine hesitancy?",
    "How do healthcare professionals address vaccine concerns in patient interactions?",
    "What are the biggest challenges that healthcare professionals face in increasing vaccine acceptance?",
    "How do healthcare professionals in Alaska perceive the impact of social media on vaccine decisions?",
    "What strategies do public health officials recommend to improve vaccine trust in Alaska?",


]

In [11]:
# =============================================================================
# BLOCK 10: Retrieval & Generation using Original T5-based Pipeline
# =============================================================================
# Load a generative model for answer synthesis (using T5-base).
generative_model = pipeline("text2text-generation", model="t5-base")

def retrieve_and_generate_t5(query, index, tweet_texts, top_k=5):
    query_embedding = embedder.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    retrieved_texts = [tweet_texts[i] for i in indices[0] if i < len(tweet_texts)]
    context = " ".join(retrieved_texts)
    prompt = f"summarize: Given the following tweets, answer the question '{query}': {context}"
    answer = generative_model(prompt, max_length=256)[0]['generated_text']
    return answer, retrieved_texts

# Test T5-based retrieval for each query.
print("===== T5-based Pipeline Answers =====")
for query in queries:
    answer, retrieved = retrieve_and_generate_t5(query, faiss_index, tweet_texts, top_k=5)
    print("\nQuery:", query)
    print("Generated Answer:", answer)
    print("Sample Retrieved Tweets:", retrieved[:3])

Device set to use cuda:0


===== T5-based Pipeline Answers =====

Query: What are the most common reasons for COVID-19 vaccine hesitancy among Alaskans?
Generated Answer: transmission of #COVID19 is still high in #Alaska and hospitals continue to be stressed .
Sample Retrieved Tweets: ["Community transmission of #COVID19 is still high in #Alaska and hospitals continue to be stressed. Please consider getting vaccinated if you haven\\'t done so already. Learn more and find a vaccine provider at https://t.co/yYSFxRUxCO https://t.co/WRtQncBNPD", "Community transmission of #COVID19 is still high in #Alaska and hospitals continue to be stressed. Please consider getting vaccinated if you haven\\'t done so already. Learn more and find a vaccine provider at https://t.co/yYSFxRUxCO https://t.co/WRtQncBNPD", 'Alaskaâ\x80\x99s COVID-19 rates dropped but remain some of the highest in the country   GOP Governor Dunleavy &amp; Alaskan mayors Matherley &amp; Bronson work to keep the virus active; no mask or vaccine mandates her

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Query: What are the biggest logistical challenges Alaskans face in getting vaccinated?
Generated Answer: #Alaskans have such a perverse sense of entitlement and individualism they can't be bothered to wear a mask + get a vaccine during a pandemic that has killed over 5 million worldwide . state health officials are asking Alaskans to get their COVID-19 vaccine booster and redouble other precautions over the coming weeks .
Sample Retrieved Tweets: ['@agnesbookbinder Oh, Iâ\x80\x99m so sorry, Agnes. In Alaska it was super easy. Low population + high vaccine hesitancy worked weirdly in our favor.', "Apparently half the US population and half of all #Alaskans have such a perverse, radicalized sense of entitlement and individualism they can\\'t be bothered to wear a mask + get a vaccine during a pandemic that has killed over 5 million worldwide.", 'State health officials are asking Alaskans to get their COVID-19 vaccine booster and redouble other precautions over the coming weeks as they b

In [12]:
# =============================================================================
# BLOCK 11: Hugging Face Login
# =============================================================================
# huggin face token
# 
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

In [13]:
# =============================================================================
# BLOCK 12: Llama-index Retrieval Pipeline
# =============================================================================
# (Make sure you have logged in: !huggingface-cli login)
import torch
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb # import bitsandbytes
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# =============================================================================
# BLOCK 13: Use HuggingFaceEmbedding for the Llama-index pipeline.
# =============================================================================

embed_model_llama = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [15]:
# =============================================================================
# BLOCK 14: Retreiver.
# =============================================================================
from llama_index.core import VectorStoreIndex, ServiceContext, Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model_llama
Settings.chunk_size = 1024

# For demonstration, we use the same tweet texts as our document corpus.
documents = [Document(text=doc) for doc in tweet_texts]

llama_index=VectorStoreIndex.from_documents(documents)


retriever = VectorIndexRetriever(index=llama_index, similarity_top_k=100)
#takes set of nodes and applies some kind of transformation, filter nodes below 0.80 similarity score
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.55)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[postprocessor])

In [16]:
print("FAISS Index Size:", faiss_index.ntotal)


FAISS Index Size: 1342


In [19]:
!pip install llama_index

In [21]:
# =============================================================================
# BLOCK 14: Compare Queries Using Both Pipelines
# =============================================================================

from llama_index.core.query_engine import RetrieverQueryEngine

print("\n===== Comparison: Llama-index Pipeline Answers =====")
for query in queries:
    print("\nQuery:", query)
    # response = llama_query_engine.query(query)
    response = query_engine.query(query)
    print("Llama-index Generated Answer:", response)
    # Optionally, show source nodes (excerpts)
    for node in response.source_nodes[:3]:
        print("Source Node:", node)


===== Comparison: Llama-index Pipeline Answers =====

Query: What are the most common reasons for COVID-19 vaccine hesitancy among Alaskans?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Llama-index Generated Answer: 
Based on the context information provided, the most common reasons for COVID-19 vaccine hesitancy among Alaskans are:

1. Lack of trust in the government and health officials: Many Alaskans seem to have a perverse sense of entitlement and individualism, which makes them unwilling to take precautions such as getting vaccinated.
2. Misinformation and disinformation: There are many false claims and conspiracy theories circulating about the COVID-19 vaccine, which may be contributing to vaccine hesitancy among Alaskans.
3. Concerns about adverse reactions: Some Alaskans may be concerned about the potential risks of the COVID-19 vaccine, such as adverse reactions, despite the fact that the vaccine has been proven to be safe and effective.
4. Resistance to medical coercion: Some Alaskans may be opposed to the idea of being forced to get vaccinated, even if it is for their own protection.
5. Lack of access: Some Alaskans may not have access to the COVID-19 vacci

In [22]:
# =============================================================================
# BLOCK 15: Format comparison in HTML Format.
# =============================================================================
import pandas as pd
from IPython.display import display, HTML
from llama_index.core.query_engine import RetrieverQueryEngine

# Create an empty list to store the results
results = []

for query in queries:
    # Get T5-based answer
    t5_answer, t5_retrieved = retrieve_and_generate_t5(query, faiss_index, tweet_texts, top_k=5)

    # Get Llama-index answer
    #llama_response = llama_query_engine.query(query)
    llama_response = query_engine.query(query)
    # For simplicity, we convert the response to string.
    llama_answer = str(llama_response).strip()

    # Append the results for this query
    results.append({
        "Query": query,
        "T5 Answer": t5_answer,
        "Llama-index Answer": llama_answer,
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Optionally, use HTML formatting for better display in Colab/Jupyter Notebook.
display(HTML(df_results.to_html(index=False)))


Query,T5 Answer,Llama-index Answer
What are the most common reasons for COVID-19 vaccine hesitancy among Alaskans?,transmission of #COVID19 is still high in #Alaska and hospitals continue to be stressed .,"Based on the context information provided, the most common reasons for COVID-19 vaccine hesitancy among Alaskans are:\n\n1. Lack of trust in the government and health officials: Many Alaskans seem to have a perverse sense of entitlement and individualism, which makes them unwilling to take precautions such as getting vaccinated.\n2. Misinformation and disinformation: There are many false claims and conspiracy theories circulating about the COVID-19 vaccine, which may be contributing to vaccine hesitancy among Alaskans.\n3. Concerns about adverse reactions: Some Alaskans may be concerned about the potential risks of the COVID-19 vaccine, such as adverse reactions, despite the fact that the vaccine has been proven to be safe and effective.\n4. Resistance to medical coercion: Some Alaskans may be opposed to the idea of being forced to get vaccinated, even if it is for their own protection.\n5. Lack of access: Some Alaskans may not have access to the COVID-19 vaccine due to various reasons such as lack of availability, lack of transportation, or lack of healthcare infrastructure.\n\nIt is important to note that these are general reasons and may not be applicable to all Alaskans. It is crucial to address these concerns through education, awareness, and access to healthcare services to increase vaccination rates in Alaska."
What are the key concerns about COVID-19 vaccine safety and effectiveness in Alaska?,state health officials are asking Alaskans to get their COVID-19 vaccine booster and redouble other precautions over the coming weeks . community transmission of #COVID19 is still high in #Alaska and hospitals continue to be stressed .,"The key concerns about COVID-19 vaccine safety and effectiveness in Alaska include:\n\n1. Vaccine hesitancy: Many people in Alaska are hesitant to get vaccinated due to various reasons such as lack of trust in the government, concerns about side effects, and beliefs that the vaccine is not effective.\n2. Low vaccination rates: Despite the high transmission rate of COVID-19 in Alaska, the vaccination rates are relatively low, which can lead to a higher risk of infection and hospitalization.\n3. Long-term safety studies: There are concerns about the long-term safety of the COVID-19 vaccine, as there have been few long-term studies conducted on its safety.\n4. Adverse reactions: Some people in Alaska have reported adverse reactions to the COVID-19 vaccine, such as allergic reactions, autoimmune disorders, and other serious side effects.\n5. Lack of data on vaccine effectiveness: There is a lack of data on the effectiveness of the COVID-19 vaccine in preventing severe illness and death, particularly among vulnerable populations such as the elderly and those with underlying health conditions.\n6. Scapegoating of unvaccinated individuals: There are concerns that unvaccinated individuals in Alaska may be scapegoated and blamed for the spread of COVID-19, leading to discrimination and stigma.\n7. Vaccine failure: There are concerns that the COVID-19 vaccine may not be effective in preventing the spread of the virus, particularly in light of the emergence of new variants.\n8. Under-reporting of adverse events: There are concerns that adverse events following COVID-19 vaccination may be under-reported, which can lead to a lack of transparency and accountability.\n9. Lack of natural immunity: There are concerns that the COVID-19 vaccine may not provide natural immunity, which can lead to a higher risk of infection and hospitalization in the long term.\n10. Mandates: There are concerns about vaccine mandates, which can lead to a loss of personal freedom and autonomy, particularly in light of the emergence of new variants."
How do past negative experiences with vaccines shape vaccine hesitancy?,'i'm not insi